In [2]:
import pandas as pd
import json
import ast
import warnings
import pyarrow as pa
import pyarrow.parquet as pq
warnings.filterwarnings('ignore')

In [3]:
steam_games = pd.read_csv('csv_limpios/steam_games.csv')
users_items = pd.read_csv('csv_limpios/user_items.csv')
user_reviews = pd.read_csv('csv_limpios/user_reviews.csv')

In [4]:
steam_games['genres'].unique()

array(['Action', 'Casual', 'Indie', 'Simulation', 'Strategy',
       'Free to Play', 'RPG', 'Sports', 'Adventure', nan, 'Racing',
       'Early Access', 'Massively Multiplayer',
       'Animation &amp; Modeling', 'Video Production', 'Utilities',
       'Web Publishing', 'Education', 'Software Training',
       'Design &amp; Illustration', 'Audio Production', 'Photo Editing',
       'Accounting'], dtype=object)

In [5]:
steam_games[steam_games['id'] == 50]

,id,title,genres,price,year
74773,50,Half-Life: Opposing Force,Action,4.99,1999


In [6]:
users_items[users_items['id'] == 50]

,user_id,items_count,steam_id,id,item_name,playtime_forever,playtime_2weeks
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0
289,js41637,888,76561198035864385,50,Half-Life: Opposing Force,0.0,0.0
1306,Riot-Punch,328,76561197963445855,50,Half-Life: Opposing Force,0.0,0.0
1632,doctr,541,76561198002099482,50,Half-Life: Opposing Force,178.0,0.0
2171,MinxIsBetterThanPotatoes,371,76561198004744620,50,Half-Life: Opposing Force,256.0,0.0
...,...,...,...,...,...,...,...
5147340,steamjke,26,76561198209116983,50,Half-Life: Opposing Force,0.0,0.0
5148476,76561198229217276,38,76561198229217276,50,Half-Life: Opposing Force,0.0,0.0
5150259,elasticgoose,42,76561198271434524,50,Half-Life: Opposing Force,0.0,0.0
5150623,943525,58,76561198283103833,50,Half-Life: Opposing Force,0.0,0.0


### Consulta PlayTimeGenre( genero : str ):
- Debe devolver año con mas horas jugadas para dicho género.

Para esta consulta se tiene que vincular el dataframe de user_items que posee el valor de horas jugadas totales (playtime_forever) con el dataframe steam_games que posee la columna generos.

In [7]:
items_genre = users_items[['id', 'item_name', 'playtime_forever']]

games_genre = steam_games[['id', 'genres', 'year']]

playTimeGenre = pd.merge(items_genre, games_genre, on='id', how='inner')

playTimeGenre

,id,item_name,playtime_forever,genres,year
0,10,Counter-Strike,6.0,Action,2000
1,10,Counter-Strike,0.0,Action,2000
2,10,Counter-Strike,0.0,Action,2000
3,10,Counter-Strike,93.0,Action,2000
4,10,Counter-Strike,108.0,Action,2000
...,...,...,...,...,...
10083386,354280,ChaosTower,164.0,Simulation,2016
10083387,433920,Aveyond 4: Shadow Of The Mist,0.0,Adventure,2016
10083388,433920,Aveyond 4: Shadow Of The Mist,0.0,Indie,2016
10083389,433920,Aveyond 4: Shadow Of The Mist,0.0,RPG,2016


In [8]:
#Especifico el género que deseas consultar
consulta = 'Indie'  #Se reemplaza con el género que consultado

#Filtro el dataFrame con la consulta
genero = playTimeGenre[playTimeGenre['genres'] == consulta]

#Agrupo  por año y suma las horas jugadas
resultados = genero.groupby('year')['playtime_forever'].sum().reset_index()

# Encuentra el año con más horas jugadas
año_max_horas = resultados.loc[resultados['playtime_forever'].idxmax()]

# Imprime el resultado
print(f"Para el género '{consulta}', el año con más horas jugadas es {int(año_max_horas['year'])} con {int(año_max_horas['playtime_forever'])} horas.")


Para el género 'Indie', el año con más horas jugadas es 2006 con 452672790 horas.


In [9]:
from fastapi import FastAPI

app = FastAPI()

def obtener_año_max_horas(consulta):
    # Filtro el DataFrame con la consulta
    genero = playTimeGenre[playTimeGenre['genres'] == consulta]

    # Agrupo por año y suma las horas jugadas
    resultados = genero.groupby('year')['playtime_forever'].sum().reset_index()

    # Encuentra el año con más horas jugadas
    año_max_horas = resultados.loc[resultados['playtime_forever'].idxmax()]

    # Devuelve el resultado como un diccionario
    return {"genero": consulta, "año_max_horas": {"año": int(año_max_horas['year']), "horas": int(año_max_horas['playtime_forever'])}}

@app.get("/consulta/{genero}", response_model=dict)
def consulta_año_max_horas(genero: str):
    resultado = obtener_año_max_horas(genero)
    return resultado

### Consulta UserForGenre( genero : str ):
- Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Para esta consulta se necesitan del dataframe users_items, user_id, id, playtime_forever, y del dataframe steam_Games, genres, id, year.

In [10]:
items_UserForGenre = users_items[['id', 'user_id', 'playtime_forever']]

games_UserForGenre = steam_games[['id', 'genres', 'year']]

UserForGenre = pd.merge(items_UserForGenre, games_UserForGenre, on='id', how='inner')

In [11]:
UserForGenre

,id,user_id,playtime_forever,genres,year
0,10,76561197970982479,6.0,Action,2000
1,10,js41637,0.0,Action,2000
2,10,Riot-Punch,0.0,Action,2000
3,10,doctr,93.0,Action,2000
4,10,corrupted_soul,108.0,Action,2000
...,...,...,...,...,...
10083386,354280,76561198107283457,164.0,Simulation,2016
10083387,433920,inven,0.0,Adventure,2016
10083388,433920,inven,0.0,Indie,2016
10083389,433920,inven,0.0,RPG,2016


In [12]:
from typing import List
from pydantic import BaseModel

In [13]:
class UsuarioMasJugador(BaseModel):
    nombre: str
    horas_acumuladas: int

class AcumulacionPorAño(BaseModel):
    año: int
    horas_acumuladas: int

class ConsultaResultado(BaseModel):
    usuario_mas_jugado: UsuarioMasJugador
    acumulacion_por_año: List[AcumulacionPorAño]

In [14]:
#Especifico el género que deseas consultar
consulta = 'Action'  #Se reemplaza con el género que consultado

#Filtro el dataFrame con la consulta
genero = UserForGenre[UserForGenre['genres'] == consulta]

# Agrupa por usuario y año y suma las horas jugadas
resultados_usuario = genero.groupby('user_id')['playtime_forever'].sum().reset_index()
usuario_mas_jugado = resultados_usuario.loc[resultados_usuario['playtime_forever'].idxmax()]

# Agrupa por año y suma las horas jugadas
resultados_año = genero.groupby('year')['playtime_forever'].sum().reset_index()

# Crea las instancias de modelos Pydantic para la respuesta
usuario_resultado = UsuarioMasJugador(nombre=str(usuario_mas_jugado['user_id']), horas_acumuladas=int(usuario_mas_jugado['playtime_forever']))
acumulacion_por_año_resultado = [AcumulacionPorAño(año=int(row['year']), horas_acumuladas=int(row['playtime_forever'])) for index, row in resultados_año.iterrows()]

# Devuelve el resultado como un diccionario
print(ConsultaResultado(usuario_mas_jugado=usuario_resultado, acumulacion_por_año=acumulacion_por_año_resultado))

usuario_mas_jugado=UsuarioMasJugador(nombre='Sp3ctre', horas_acumuladas=1699307) acumulacion_por_año=[AcumulacionPorAño(año=1983, horas_acumuladas=3582), AcumulacionPorAño(año=1984, horas_acumuladas=384), AcumulacionPorAño(año=1988, horas_acumuladas=16243), AcumulacionPorAño(año=1989, horas_acumuladas=607), AcumulacionPorAño(año=1990, horas_acumuladas=18787), AcumulacionPorAño(año=1991, horas_acumuladas=2502), AcumulacionPorAño(año=1992, horas_acumuladas=1925), AcumulacionPorAño(año=1993, horas_acumuladas=24663), AcumulacionPorAño(año=1994, horas_acumuladas=121057), AcumulacionPorAño(año=1995, horas_acumuladas=217813), AcumulacionPorAño(año=1996, horas_acumuladas=70061), AcumulacionPorAño(año=1997, horas_acumuladas=363033), AcumulacionPorAño(año=1998, horas_acumuladas=2935053), AcumulacionPorAño(año=1999, horas_acumuladas=2735338), AcumulacionPorAño(año=2000, horas_acumuladas=18643971), AcumulacionPorAño(año=2001, horas_acumuladas=1337303), AcumulacionPorAño(año=2002, horas_acumuladas=

In [15]:
def obtener_año_y_usuario_max_horas(consulta):

    #Filtro el dataFrame con la consulta
    genero = UserForGenre[UserForGenre['genres'] == consulta]

    # Agrupa por usuario y año y suma las horas jugadas
    resultados_usuario = genero.groupby('user_id')['playtime_forever'].sum().reset_index()
    usuario_mas_jugado = resultados_usuario.loc[resultados_usuario['playtime_forever'].idxmax()]

    # Agrupa por año y suma las horas jugadas
    resultados_año = genero.groupby('year')['playtime_forever'].sum().reset_index()

    # Crea las instancias de modelos Pydantic para la respuesta
    usuario_resultado = UsuarioMasJugador(nombre=str(usuario_mas_jugado['user_id']), horas_acumuladas=int(usuario_mas_jugado['playtime_forever']))
    acumulacion_por_año_resultado = [AcumulacionPorAño(año=int(row['year']), horas_acumuladas=int(row['playtime_forever'])) for index, row in resultados_año.iterrows()]

    # Devuelve el resultado como un diccionario
    return ConsultaResultado(usuario_mas_jugado=usuario_resultado, acumulacion_por_año=acumulacion_por_año_resultado)


In [16]:
# Segunda consulta de la API

@app.get("/consulta2/{genero}", response_model=ConsultaResultado)
def consulta_año_y_usuario_max_horas(genero: str):
    resultado = obtener_año_y_usuario_max_horas(genero)
    return resultado

### Consulta UsersRecommend( año : int ):
- Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Para esta consulta se necesita el dataframe user_reviews completo y del dataframe steam_games solo las columnas id y title

In [17]:
#items_UsersRecommend = user_reviews[['id', 'user_id', 'review_year', 'sentiment_analysis']]

games_UsersRecommend = steam_games[['id', 'title']]

UsersRecommend = pd.merge(user_reviews, games_UsersRecommend, on='id', how='inner')

In [18]:
UsersRecommend

,user_id,id,recommend,review_year,sentiment_analysis,title
0,76561197970982479,1250,True,2011,2,Killing Floor
1,death-hunter,1250,True,2015,2,Killing Floor
2,DJKamBer,1250,True,2013,0,Killing Floor
3,diego9031,1250,True,2015,1,Killing Floor
4,76561198081962345,1250,True,2014,1,Killing Floor
...,...,...,...,...,...,...
127789,MeloncraftLP,262850,True,2015,1,The Journey Down: Chapter Two
127790,vinquility,431510,True,2015,2,Mystic Destinies: Serendipity of Aeons
127791,vinquility,431510,True,2015,2,Mystic Destinies: Serendipity of Aeons
127792,vinquility,431510,True,2015,2,Mystic Destinies: Serendipity of Aeons


In [19]:
# Testeo de año a ingresar
consulta = 2015

In [20]:
# Primer filtro utilizando el año ingresado
UsersRecommend_filtrado_anio = UsersRecommend[UsersRecommend['review_year'] == consulta]

# Segundo filtro donde contemplo solo las filas que cumplen con ambos booleanos, ser False en la columna recommend y tener valor 0 en la columna sentiment_analysis
UsersRecommend_filtrado = UsersRecommend_filtrado_anio[(UsersRecommend_filtrado_anio['recommend'] == True) & (UsersRecommend_filtrado_anio['sentiment_analysis'].isin([1,2]))]

# Agrupar por id de juego y sumar las recomendaciones
UsersRecommend_filtrado_anio = UsersRecommend_filtrado[UsersRecommend_filtrado['review_year'] == consulta]

top_games = UsersRecommend_filtrado_anio.groupby('id').agg({'title': 'first', 'recommend': 'sum'}).reset_index()

top_games_sorted = top_games.sort_values(by='recommend', ascending=False)

top3Games = top_games_sorted.head(3)

In [21]:
top3Games

,id,title,recommend
22,730,Counter-Strike: Global Offensive,1527
1089,304930,Unturned,1430
16,440,Team Fortress 2,1378


### Consulta UsersNotRecommend( año : int ):
- Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Para esta consulta se necesita el dataframe user_reviews completo y del dataframe steam_games solo las columnas id y title

In [22]:
# Testeo de año a ingresar
consulta = 2015

In [23]:
# Primer filtro utilizando el año ingresado
UsersRecommend_filtrado_anio = UsersRecommend[UsersRecommend['review_year'] == consulta]

# Segundo filtro donde contemplo solo las filas que cumplen con ambos booleanos, ser False en la columna recommend y tener valor 0 en la columna sentiment_analysis
UsersRecommend_filtrado = UsersRecommend_filtrado_anio[(UsersRecommend_filtrado_anio['recommend'] == False) & (UsersRecommend_filtrado_anio['sentiment_analysis'].isin([0]))]

# Agrupar por id de juego y sumar las recomendaciones
UsersRecommend_filtrado_anio = UsersRecommend_filtrado[UsersRecommend_filtrado['review_year'] == consulta]

top_games = UsersRecommend_filtrado_anio.groupby('id').agg({'title': 'first', 'recommend': 'sum'}).reset_index()

top_games_sorted = top_games.sort_values(by='recommend', ascending=True)

top3Games = top_games_sorted.head(3)

In [24]:
top3Games

,id,title,recommend
0,20,Team Fortress Classic,0
342,319510,Five Nights at Freddy's,0
341,318791,Call of Duty®: Advanced Warfare - Havoc,0


### Consulta sentiment_analysis( año : int ):
- Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Para esta consulta se necesita el dataframe...

In [25]:
user_reviews

,user_id,id,recommend,review_year,sentiment_analysis
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,2
...,...,...,...,...,...
58426,76561198312638244,70,True,2015,2
58427,76561198312638244,362890,True,2015,2
58428,LydiaMorley,273110,True,2015,2
58429,LydiaMorley,730,True,2015,2


In [26]:
games_sentiment = steam_games[['id', 'title', 'year']]

user_sentiment = user_reviews[['id', 'user_id', 'sentiment_analysis' ]]

sentiment = pd.merge(user_sentiment, games_sentiment, on='id', how='inner')

In [27]:
sentiment

,id,user_id,sentiment_analysis,title,year
0,1250,76561197970982479,2,Killing Floor,2009
1,1250,death-hunter,2,Killing Floor,2009
2,1250,DJKamBer,0,Killing Floor,2009
3,1250,diego9031,1,Killing Floor,2009
4,1250,76561198081962345,1,Killing Floor,2009
...,...,...,...,...,...
127789,262850,MeloncraftLP,1,The Journey Down: Chapter Two,2014
127790,431510,vinquility,2,Mystic Destinies: Serendipity of Aeons,2016
127791,431510,vinquility,2,Mystic Destinies: Serendipity of Aeons,2016
127792,431510,vinquility,2,Mystic Destinies: Serendipity of Aeons,2016


In [28]:
# Testeo de año a ingresar
consulta = 2015

In [29]:
df_filtrado = sentiment[sentiment['year'] == consulta]

    # Contar la cantidad de registros por categoría de análisis de sentimiento
cantidad_sentimientos = df_filtrado['sentiment_analysis'].value_counts().to_dict()
cantidad_sentimientos

{2: 11617, 1: 3780, 0: 3557}

In [30]:
# Filtrar por año específico
df_filtrado = sentiment[sentiment['year'] == consulta]

# Contar la cantidad de registros por categoría de análisis de sentimiento
cantidad_sentimientos = df_filtrado['sentiment_analysis'].value_counts()

# Mapear los valores a los nombres deseados
cantidad_sentimientos_nombres = {
    "Positivo": cantidad_sentimientos.get(2, 0),
    "Neutral": cantidad_sentimientos.get(1, 0),
    "Negativo": cantidad_sentimientos.get(0, 0)
}

# Devolver el resultado
cantidad_sentimientos_nombres

{'Positivo': 11617, 'Neutral': 3780, 'Negativo': 3557}

In [31]:
# Filtrar por año específico
df_filtrado = sentiment[sentiment['year'] == consulta]
df_filtrado

,id,user_id,sentiment_analysis,title,year
1059,370360,evcentric,2,TIS-100,2015
1060,370360,evcentric,2,TIS-100,2015
16699,252950,76561198156664158,2,Rocket League®,2015
16700,252950,76561198156664158,2,Rocket League®,2015
16701,252950,76561198156664158,2,Rocket League®,2015
...,...,...,...,...,...
127768,369000,zigzagsoft,2,Pongo,2015
127772,367780,laislabonita75,2,Aero's Quest,2015
127773,367780,laislabonita75,2,Aero's Quest,2015
127774,367780,evilindiegaming,2,Aero's Quest,2015


In [32]:
# Contar la cantidad de registros por categoría de análisis de sentimiento
cantidad_sentimientos = df_filtrado['sentiment_analysis'].value_counts().to_dict()
cantidad_sentimientos

{2: 11617, 1: 3780, 0: 3557}

In [33]:
cantidad_sentimientos_nombres = {
        "Positivo": cantidad_sentimientos.get(2, 0),
        "Neutral": cantidad_sentimientos.get(1, 0),
        "Negativo": cantidad_sentimientos.get(0, 0)        
    }

In [34]:
cantidad_sentimientos_nombres

{'Positivo': 11617, 'Neutral': 3780, 'Negativo': 3557}

In [35]:
# Filtrar por año específico
df_filtrado = sentiment[sentiment['year'] == consulta]

# Contar la cantidad de registros por categoría de análisis de sentimiento
cantidad_sentimientos = df_filtrado['sentiment_analysis'].value_counts().to_dict()

# Devolver el resultado
cantidad_sentimientos

{2: 11617, 1: 3780, 0: 3557}